In [1]:
data_var = '2023-12-04'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1856,2023-12-04,Grécia Liga De Basquete,15:15,PAOK,Kolossos Rhodes,1.35,3.16,155.5,1.87,1.85,-7.5,1.95,1.76,SGXX8Ez5,0.740741,0.316456,0.534759,0.540541,0.057196,248.084,237.525957,0.957442,1.2,1.643168,1.369306,233.80,3.680,3.896351,1.058791,-25.0,255.804,162.107433,0.633717,1.2,1.643168,1.369306,514.80,3.288,2.104571,0.640076,-8.0,70,78,3.34,6.60,0.000,0.000,0.567567,0.007603,0.072426,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1857,2023-12-04,Suécia Basketligan,15:04,BC Lulea,Norrkoping,2.02,1.72,172.5,1.87,1.79,1.5,1.87,1.83,YwpAMLvn,0.495050,0.581395,0.534759,0.558659,0.076445,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,105,93,1.18,1.31,0.000,0.000,0.113440,0.030912,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1858,2023-12-04,Israel Superliga,12:00,Hapoel Eilat,Bnei Herzliya,1.85,1.89,165.5,1.86,1.80,-1.5,1.98,1.73,rRlgC003,0.540541,0.529101,0.537634,0.555556,0.069641,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,70,0.00,1.42,0.000,0.000,0.015125,0.023184,0.095297,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1859,2023-12-04,Israel Superliga,16:00,Ironi Kiryat Ata,H. Haifa,1.39,2.89,169.5,1.89,1.85,-7.5,2.10,1.65,Wl6VhCar,0.719424,0.346021,0.529101,0.540541,0.065445,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,82,0.00,3.03,0.000,0.000,0.495636,0.015125,0.169706,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1860,2023-12-04,Libano Divisão 1,11:45,Hoops,Al-Riyadi,7.50,1.07,165.5,1.85,1.85,13.5,1.84,1.86,jZYhGXAb,0.133333,0.934579,0.540541,0.540541,0.067913,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,78,97,2.45,1.13,0.000,0.000,1.061073,0.000000,0.007644,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1861,2023-12-04,Lituânia Lkl,13:50,Nevezis,Gargzdai,1.62,2.20,169.5,1.84,1.86,-3.5,1.95,1.76,vH0YKYHd,0.617284,0.454545,0.543478,0.537634,0.071829,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,90,66,1.70,2.84,0.000,0.000,0.214724,0.007644,0.072426,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1862,2023-12-04,Lituânia Nkl,13:30,Suduva-Mantinga,Vilkaviskis,1.17,4.62,159.5,1.82,1.84,-12.5,2.05,1.69,AD8kFCV0,0.854701,0.216450,0.549451,0.543478,0.071151,128.830,58.380258,0.453157,3.0,0.000000,0.000000,99.76,1.580,0.714248,0.452056,74.0,129.202,27.466431,0.212585,2.4,1.341641,0.559017,103.70,1.476,0.226782,0.153646,28.0,86,85,1.16,1.22,0.000,0.000,0.842666,0.007728,0.136128,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
1863,2023-12-04,Romênia Liga Nacional Feminina,14:00,CSM Constanta F,Arad F,1.32,3.20,138.5,1.83,1.83,-8.5,2.04,1.69,AwS6rQ1h,0.757576,0.312500,0.546448,0.546448,0.070076,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.00

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    
    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
1857,15:04,Suécia Basketligan,BC Lulea,Norrkoping,2.02,Back Home
1858,12:00,Israel Superliga,Hapoel Eilat,Bnei Herzliya,1.85,Back Home
1861,13:50,Lituânia Lkl,Nevezis,Gargzdai,1.62,Back Home
1868,15:00,Eslovênia Liga Nova Kbm,Podcetrtek,LTH Castings,1.65,Back Home
1872,22:00,Eua Ncaa,North Dakota St.,San Jose State,1.71,Back Home
1886,08:35,China Cba,Beijing Royal Fighters,Beijing,1.65,Back Home
1898,14:00,Europa Liga Aba,Studentski Centar,Mega Basket,1.69,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
1886,08:35,China Cba,Beijing Royal Fighters,Beijing,1.9,Back Home
